# Create NoSQL tables in Apache Cassandra for 3 queries

This notebook retrieves data from the event data folder and creates 3 queries and tables to analyze the data. All tables and queries are run in Apache Cassandra.

### Keyspace: 
   - Name: "events"
   - Cassandra instance: 127.0.0.1

### Queries:

1. Query 1
    - Returns: A single item from one session, gives details on the artist, song, and length
    - Table: artist_songs
    - Primary key: sessionid (partition), iteminsession (cluster)  
        - query is filtered on both sessionid and iteminsession. As sessionid acts as a parent to iteminsession, the former is used as the partition key so all session items are grouped together in query
    - Table Columns: artist, song, length, sessionid, iteminsession
        
2. Query 2
    - Returns: Shows all items in a single session from user Sylvie Cruz
    - Table: user_session
    - Primary key: userid, sessionid (partition), iteminsession (cluster)  
        - query is filtered on both userid and sessionid and needs to be further sorted by iteminsession. The three create a unique combination in the data.
    - Table Columns: artist, song, firstname, lastname, userid, sessionid, iteminsession  
    
3. Query 3
    - Returns: Shows any user who has ever listened to the song "All Hands On Me"
    - Table: user_song
    - Primary key: song (partition), sessionid, iteminsession (cluster) 
        - query is filtered song. Both sessionid and iteminsession ensure that each key is unique.
    - Table Columns: song, firstname, lastname, sessionid, iteminsession

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [6]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [7]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data' 
# os.getcwd returns the path of the current working directory

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

# Run below scripts to create the tables and queries required to anazlyze the data

#### Creating a Cluster

In [9]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [13]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS events
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 } """)
except Exception as e:
    print(e)

#### Set Keyspace

In [14]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [34]:
#Create table for query 1

query = "CREATE TABLE IF NOT EXISTS artist_songs"
query = query + "(artist text, song text, length float, sessionid int, \
                    iteminsession int, PRIMARY KEY (sessionid, iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [35]:
# Read event_datafile_new.csv and insert relevant records into user_session table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Create INSERT statements into the `query` variable
        query = "INSERT INTO artist_songs (artist, song, length, sessionid, iteminsession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
# Run query 1 to check data was entered into table correctly

query = "SELECT artist, song, length FROM artist_songs WHERE sessionid = 338 AND iteminsession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Insert rows into dataframe and print to show results

df = pd.DataFrame(list(rows))
print(df)

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


### Query 2: Return name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [26]:
# Create table for query 2
query = "CREATE TABLE IF NOT EXISTS user_session"
query = query + "(artist text, song text, firstname text, lastname text, \
                userid int, sessionid int, iteminsession int, \
                PRIMARY KEY ((userid, sessionid), iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Insert csv data into second query table

In [27]:
# Read event_datafile_new.csv and insert relevant records into user_session table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Create INSERT statements into the `query` variable
        query = "INSERT INTO user_session (artist, song, firstName, lastName, userid, sessionid, iteminsession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

#### Create select statement for query 2 to verify data has been entered

In [28]:
# Run query 2 to check data was entered into table correctly

query = "SELECT artist, song, firstName, lastName FROM user_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Insert rows into dataframe and print to show results

df = pd.DataFrame(list(rows))
print(df)

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


### Query 3: Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [30]:
# Create table for query 3
query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + "(song text, firstname text, lastname text, sessionid int, iteminsession int, PRIMARY KEY ((song), sessionid, iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                   
                    

#### Insert csv data into third query table

In [32]:
# Read event_datafile_new.csv and insert relevant records into user_song table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Create INSERT statements into the `query` variable
        query = "INSERT INTO user_song (song, firstname, lastname, sessionid, iteminsession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[8]), int(line[3])))

#### Create select statement for query 3 to verify data has been entered

In [33]:
# Run query 3 to check data was entered into table correctly

query = "SELECT firstname, lastname FROM user_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Insert rows into dataframe and print to show results

df = pd.DataFrame(list(rows))
print(df)

    firstname lastname
0        Sara  Johnson
1  Jacqueline    Lynch
2       Tegan   Levine


### Drop the tables before closing out the sessions

In [36]:
# Drop the table before closing out the sessions

session.execute("DROP TABLE artist_songs")
session.execute("DROP TABLE user_session")
session.execute("DROP TABLE user_song")


### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()